<a href="https://colab.research.google.com/github/Srijani-coder/Learning_ML_Path/blob/main/factcheck_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install requests-html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 17.0 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=3043e4af30e6749c5752a7fa148c44ca2c815b0401de5973c661622593d8816a
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [2]:
from requests_html import HTMLSession

# establish a session
session = HTMLSession()



In [3]:
def get_search_url(url):
  root_url = "https://mediabiasfactcheck.com/?s="
  sum_url = root_url + url
  resp = session.get(sum_url)
  return resp

In [4]:
import re

In [5]:
def get_bias_search_url_tool(url):
  response = get_search_url(url)
  bias_links = response.html.find("a", containing = "bias")
  pattern = r"<Element 'a' href='https://mediabiasfactcheck.com/([^/]+)/' title='([^']+) Bias and Credibility' rel=\('bookmark',\)>"
  pattern2 = r"<Element 'a' href='https://mediabiasfactcheck.com/([^/]+)/' title='([^']+)(?: Bias| Biased)' rel=\('bookmark',\)>"

  return_final = []

  for x in bias_links:
    return_to_be =[]
    biasdict = None
    url_bias_dict = None
    biascredict = None
    url_biascred_dict = None

    tex = str(x)
    match1 = re.search(pattern,tex)
    match2 = re.search(pattern2,tex)
    if match2:
      # Access the captured groups using match.group(1) and match.group(2)
      anything = match2.group(1)
      url_bias = 'https://mediabiasfactcheck.com/'+anything+'/'
      print(url_bias)
      anything_title = match2.group(2)
      print(f"Matched 'anything': {anything}")
      print(f"Matched 'Anything' (title): {anything_title}")
      biasdict = {"bias": anything_title}
      url_bias_dict = {"search_bias_url":url_bias}
    elif match1:
      anything2 = match1.group(1)
      url_biascred = 'https://mediabiasfactcheck.com/'+anything2+'/'
      print(url_biascred)
      anything_title2 = match1.group(2)
      print(f"Matched 'anything': {anything2}")
      print(f"Matched 'Anything' (title): {anything_title2}")
      biascredict = {"biascred": anything2}
      url_biascred_dict = {"search_biascred_url":url_biascred}

    return_to_be.append(url_biascred_dict)
    return_to_be.append(biascredict)
    return_to_be.append(biasdict)
    return_to_be.append(url_bias_dict)

    for item in return_to_be:
      if item is not None:
        return_final.append(return_to_be)
        break

  result = []
  for it in return_final:
    list_be = it
    for tm in list_be:
      if tm is not None:
        result.append(tm)

  return result

def get_bias_credibility_url(result_dict_tools):
  for rs in result_dict_tools:
    if 'search_biascred_url' in rs.keys():
      url = rs['search_biascred_url']
      if url is not None:
        print("search_biascred_url:"+ url)
        break
    else:
      url = None

  return url

def get_bias_url(result_dict_tools):
  for rs in result_dict_tools:
    if 'search_bias_url' in rs.keys():
      url = rs['search_bias_url']
      if url is not None:
        print("search_bias_url:"+ url)
        break
    else:
      url = None

  return url







In [6]:
import re

In [7]:
def search_for_detailed_report(response):
  pattern3 = r"Bias Rating:([^/]+)"
  pattern4 = r"Factual Reporting:([^/]+)"
  resptext = response.html.text
  match3 = re.search(pattern3,resptext)
  match4 = re.search(pattern4,resptext)
  if (match3 is not None) & (match4 is not None):
    list_match3 = match3.group(1).split("\n")
    list_match4 = match4.group(1).split("\n")
    flag_d = True
  else:
    list_match3 = [None]
    list_match4 = [None]
    flag_d = False
  return {"bias_rating":list_match3[0],"fact_report":list_match4[0],"report_flag": flag_d}


In [8]:
def collect_href_url(html_text):
  href_pattern = r"href='([^']+)'"
  # Use re.search to find the first match in the HTML text
  match = re.search(href_pattern, str(html_text))
  if match:
    href_value = match.group(1)
  else:
    href_value = None
  return href_value

In [66]:
import re

def search_in_read_more(bias_link):
    url_tree1 = bias_link
    resp_tree1 = session.get(url_tree1)
    rm_link = resp_tree1.html.find("a", containing="read more")

    if rm_link:
        url_read_more = collect_href_url(rm_link)
        resp_rm = session.get(url_read_more)
        detailed_dict = search_for_detailed_report(resp_rm)

        if detailed_dict["report_flag"] is False:
            links = resp_rm.html.find("a")
            link_pattern = r"<Element 'a' href='https://mediabiasfactcheck.com/([^/]+)/'>"
            matches_link = [re.search(link_pattern, str(l)).group(1) for l in links if re.search(link_pattern, str(l))]

            remove_words = ['left-vs-right-bias-how-we-rate-the-bias-of-media-sources', 'support-media-bias-fact-check', 'filtered-search', 'presidential-election-2024-candidates-bias-ratings', 'fair-use-policy', 'funding', 'about', 'contact']
            matches_link = [item for item in matches_link if item not in remove_words]

            if matches_link:
                item_str = str(matches_link[0])
                url_tree2 = 'https://mediabiasfactcheck.com/' + item_str + '/'
                resp_tree2 = session.get(url_tree2)
                detail_report_dict = search_for_detailed_report(resp_tree2)
            else:
                detail_report_dict = None
        else:
            detail_report_dict = detailed_dict

        return detail_report_dict

    else:
      links = resp_tree1.html.find("a")
      link_pattern = r"<Element 'a' href='https://mediabiasfactcheck.com/([^/]+)/'>"
      matches_link = [re.search(link_pattern, str(l)).group(1) for l in links if re.search(link_pattern, str(l))]
      remove_words = ['left-vs-right-bias-how-we-rate-the-bias-of-media-sources', 'support-media-bias-fact-check', 'filtered-search', 'presidential-election-2024-candidates-bias-ratings', 'fair-use-policy', 'funding', 'about', 'contact']
      matches_link = [item for item in matches_link if item not in remove_words]
      if matches_link:
        item_str = str(matches_link[0])
        url_tree2 = 'https://mediabiasfactcheck.com/' + item_str + '/'
        resp_tree2 = session.get(url_tree2)
        detail_report_dict = search_for_detailed_report(resp_tree2)
      else:
        detail_report_dict = None

    return detail_report_dict






In [10]:
def find_detailed_report(url):
  get_tools = get_bias_search_url_tool(url)

  url_bias = get_bias_url(get_tools)
  url_biascred = get_bias_credibility_url(get_tools)
  if (url_bias is None) & (url_biascred is None):
    exit(0)
  if url_biascred is not None:
    sep3 = session.get(url_biascred)
    print(sep3)
    factcheck_report_trial1 = search_for_detailed_report(sep3)
    if factcheck_report_trial1["report_flag"] == True:
      print(factcheck_report_trial1)
      print("factcheck successful at trial1")
      return factcheck_report_trial1
      exit(0)
    else:
      factcheck_report_trial2 = search_in_read_more(url_biascred)
      if factcheck_report_trial2 is not None:
        print("factcheck successful at trial2")
        return factcheck_report_trial2
        exit(0)
  if url_bias is not None:
    sep4 = session.get(url_bias)
    factcheck_report_trial3 = search_for_detailed_report(sep4)
    if factcheck_report_trial3["report_flag"] == True:
      print("factcheck successful at trial3")
      return factcheck_report_trial3
      exit(0)
    else:
      factcheck_report_trial4 = search_in_read_more(url_bias)
      if factcheck_report_trial4["report_flag"] == True:
        print("factcheck successful at trial4")
        return  factcheck_report_trial4
        exit(0)



In [11]:
news_url = []
with open('/content/sample_data/news_urls.txt','r+') as file:
  for line in file:
    news_url.append(line)


In [12]:
news_url.remove('\n')

In [13]:
urls = []
for item in news_url:
  if item == '\n':
    news_url.remove(item)
  else:
    it_st = str(item)
    it_st = it_st.replace('\n','')
    it_st = it_st.replace('\t','')
    it_st = it_st.replace(' ','')
    urls.append(it_st)

In [15]:
print(len(urls))

cnt = 0
for ur in urls:
  if ur is None:
    cnt = cnt + 1

print(cnt)



339
0


In [16]:
rt = find_detailed_report(urls[0])

https://mediabiasfactcheck.com/bbc/
Matched 'anything': bbc
Matched 'Anything' (title): BBC –
https://mediabiasfactcheck.com/leftcenter/
Matched 'anything': leftcenter
Matched 'Anything' (title): Left-Center
search_bias_url:https://mediabiasfactcheck.com/leftcenter/
search_biascred_url:https://mediabiasfactcheck.com/bbc/
<Response [200]>
{'bias_rating': ' LEFT-CENTER', 'fact_report': ' HIGH', 'report_flag': True}
factcheck successful at trial1


In [22]:
def get_factreport_extracted(url):
  rt = None
  error_url = None
  if url is not None:
      try:
        rt = find_detailed_report(url)
        print(rt)
      except:
        print("error occured")
        error_url = url

  return{ 'fact_dict': rt, 'error_url': url}

In [18]:
import pandas as pd
dict_urls = {'url': urls}
df = pd.DataFrame(dict_urls)
print(df.head(5))

                         url
0       https://www.bbc.com/
1  https://www.msn.com/en-au
2        https://www.cnn.com
3   https://edition.cnn.com/
4   https://www.nytimes.com/


In [23]:
df['report_dict'] = df['url'].apply(get_factreport_extracted)

Streaming output truncated to the last 5000 lines.
Matched 'Anything' (title): 911Truth.org –
https://mediabiasfactcheck.com/debate-org-bias/
Matched 'anything': debate-org-bias
Matched 'Anything' (title): Debate.org –
https://mediabiasfactcheck.com/crossexamined-org-bias/
Matched 'anything': crossexamined-org-bias
Matched 'Anything' (title): CrossExamined.org –
https://mediabiasfactcheck.com/nutritionfacts-org/
Matched 'anything': nutritionfacts-org
Matched 'Anything' (title): NutritionFacts.org –
https://mediabiasfactcheck.com/votevets-org/
Matched 'anything': votevets-org
Matched 'Anything' (title): VoteVets.org –
https://mediabiasfactcheck.com/bustatroll-org/
Matched 'anything': bustatroll-org
Matched 'Anything' (title): Bustatroll.org –
https://mediabiasfactcheck.com/jehovahs-witnesses-jw-org/
Matched 'anything': jehovahs-witnesses-jw-org
Matched 'Anything' (title): Jehovah’s Witnesses (jw.org) –
https://mediabiasfactcheck.com/countercurrents-org/
Matched 'anything': countercurren

In [24]:
len(df)

339

In [26]:
df.tail(5)

,url,report_dict
334,https://twitter.com/hitsradiouk/status/1717586...,{'fact_dict': {'bias_rating': ' EXTREME RIGHT'...
335,https://en.wikipedia.org/wiki/Kiss_Network,{'fact_dict': {'bias_rating': ' EXTREME RIGHT'...
336,https://www.lbcexpress.com/,"{'fact_dict': {'bias_rating': ' RIGHT', 'fact_..."
337,https://www.lbc.co.uk/news/,"{'fact_dict': {'bias_rating': ' RIGHT-CENTER',..."
338,https://planetradio.co.uk/scala-radio/,"{'fact_dict': None, 'error_url': 'https://plan..."


In [27]:
report_dict_list = list(df['report_dict'])
err_list = []
for item in report_dict_list:
  if item['fact_dict'] == None:
    err = item['error_url']
    err_list.append(err)


In [30]:
len(err_list)

32

In [37]:
res_bias_search = []
for url in err_list:
  res = get_bias_search_url_tool(url)
  res_bias_search.append(res)

https://mediabiasfactcheck.com/left/
Matched 'anything': left
Matched 'Anything' (title): Left
https://mediabiasfactcheck.com/right/
Matched 'anything': right
Matched 'Anything' (title): Right
https://mediabiasfactcheck.com/70-news/
Matched 'anything': 70-news
Matched 'Anything' (title): 70 News (Fake) –
https://mediabiasfactcheck.com/health-and-money-news/
Matched 'anything': health-and-money-news
Matched 'Anything' (title): Health and Money News –
https://mediabiasfactcheck.com/ace-flashman/
Matched 'anything': ace-flashman
Matched 'Anything' (title): Ace Flashman –
https://mediabiasfactcheck.com/atavist-magazine/
Matched 'anything': atavist-magazine
Matched 'Anything' (title): Atavist Magazine –
https://mediabiasfactcheck.com/atomic-monkey/
Matched 'anything': atomic-monkey
Matched 'Anything' (title): Atomic Monkey –
https://mediabiasfactcheck.com/native-americans/
Matched 'anything': native-americans
Matched 'Anything' (title): Native Americans –
https://mediabiasfactcheck.com/atta

In [46]:
len(res_bias_search[5])

0

In [49]:
from numpy.core.multiarray import empty
cnt = 0
search_link_list = []
bias_for_read_more = []
biascred_list = []
index_list = []
for item in res_bias_search:
  print(cnt)

  if len(item) > 0:
    search_link = None
    length = len(item)
    item_dict = item[length - 1]
    if 'search_bias_url' in item_dict.keys():
      search_link = item_dict['search_bias_url']
      search_dict ={'err_list_index': cnt , 'search_link': search_link}
      bias_for_read_more.append(search_dict)
    elif 'bias' in item_dict.keys():
      bias_val = item_dict['bias']
      bias_val = bias_val.replace('-','')
      bias_val = bias_val.replace(' ','')
      bias_val = bias_val.lower()
      search_link = 'https://mediabiasfactcheck.com/'+bias_val+'/'
      search_dict ={'err_list_index': cnt , 'search_link': search_link}
      bias_for_read_more.append(search_dict)
    elif 'search_biascred_url' in item_dict.keys():
      search_link = item_dict['search_biascred_url']
      search_dict ={'err_list_index': cnt , 'search_link': search_link}
      biascred_list.append(search_dict)
    elif 'biascred' in item_dict.keys():
      biascred = item_dict['biascred']
      search_link = 'https://mediabiasfactcheck.com/'+biascred+'/'
      search_dict ={'err_list_index': cnt , 'search_link': search_link}
      biascred_list.append(search_dict)
    else:
      print(item_dict.keys())


    cnt = cnt +1












0
1
2
3
4
5
5
6
7
8
9
9
10
11
12
12
13
13
14
15
16
17
18
19
20
20
21
21
22
23
24
24


In [69]:
df_list = []
cnt = 0
for item in bias_for_read_more:
  print(cnt)
  try:
    rt = find_detailed_report(item['search_link'])
    print(rt)
    ind = item['err_list_index']
    url = err_list[ind]
    report_dict = {'fact_dict': rt , 'error_url': url}
    df_dict = {'url':url,'report_dict':report_dict}
    df_list.append(df_dict)
  except:
    print('error index:{cnt}')
  cnt = cnt + 1




0
https://mediabiasfactcheck.com/newsbusters/
Matched 'anything': newsbusters
Matched 'Anything' (title): Newsbusters –
search_biascred_url:https://mediabiasfactcheck.com/newsbusters/
<Response [200]>
{'bias_rating': ' RIGHT', 'fact_report': ' MIXED', 'report_flag': True}
factcheck successful at trial1
{'bias_rating': ' RIGHT', 'fact_report': ' MIXED', 'report_flag': True}
1
https://mediabiasfactcheck.com/deplorable-movement-2020/
Matched 'anything': deplorable-movement-2020
Matched 'Anything' (title): Deplorable Movement 2020 –
https://mediabiasfactcheck.com/newsbusters/
Matched 'anything': newsbusters
Matched 'Anything' (title): Newsbusters –
https://mediabiasfactcheck.com/native-americans/
Matched 'anything': native-americans
Matched 'Anything' (title): Native Americans –
search_biascred_url:https://mediabiasfactcheck.com/deplorable-movement-2020/
<Response [200]>
factcheck successful at trial2
{'bias_rating': None, 'fact_report': None, 'report_flag': False}
2
https://mediabiasfactc

In [70]:
df_list2 = []
cnt = 0
for item in biascred_list:
  print(cnt)
  try:
    resp = session.get(item['search_link'])
    rt = search_for_detailed_report(resp)
    print(rt)
    ind = item['err_list_index']
    url = err_list[ind]
    report_dict = {'fact_dict': rt , 'error_url': url}
    df_dict = {'url':url,'report_dict':report_dict}
    df_list2.append(df_dict)
  except:
    print('error index:{cnt}')
  cnt = cnt + 1




0
{'bias_rating': ' LEFT', 'fact_report': ' MIXED', 'report_flag': True}
1
{'bias_rating': ' RIGHT CONSPIRACY- PSEUDOSCIENCE', 'fact_report': ' VERY LOW', 'report_flag': True}
2
{'bias_rating': ' RIGHT-CENTER', 'fact_report': ' MIXED', 'report_flag': True}
3
{'bias_rating': ' LEAST BIASED', 'fact_report': ' MIXED', 'report_flag': True}
4
{'bias_rating': ' PSEUDOSCIENCE', 'fact_report': ' LOW', 'report_flag': True}
5
{'bias_rating': ' LEAST BIASED', 'fact_report': ' MIXED', 'report_flag': True}
6
{'bias_rating': ' PSEUDOSCIENCE', 'fact_report': ' LOW', 'report_flag': True}
7
{'bias_rating': ' LEAST BIASED', 'fact_report': ' MIXED', 'report_flag': True}
8
{'bias_rating': ' PSEUDOSCIENCE', 'fact_report': ' LOW', 'report_flag': True}


In [71]:
df_list2

[{'url': 'https://70newsonline.wordpress.com/',
  'report_dict': {'fact_dict': {'bias_rating': ' LEFT',
    'fact_report': ' MIXED',
    'report_flag': True},
   'error_url': 'https://70newsonline.wordpress.com/'}},
 {'url': 'https://banned.video/',
  'report_dict': {'fact_dict': {'bias_rating': ' RIGHT CONSPIRACY- PSEUDOSCIENCE',
    'fact_report': ' VERY LOW',
    'report_flag': True},
   'error_url': 'https://banned.video/'}},
 {'url': 'https://beforeitsnews.com/',
  'report_dict': {'fact_dict': {'bias_rating': ' RIGHT-CENTER',
    'fact_report': ' MIXED',
    'report_flag': True},
   'error_url': 'https://beforeitsnews.com/'}},
 {'url': 'http://www.businesspost.ie/',
  'report_dict': {'fact_dict': {'bias_rating': ' LEAST BIASED',
    'fact_report': ' MIXED',
    'report_flag': True},
   'error_url': 'http://www.businesspost.ie/'}},
 {'url': 'https://empirenews.net/',
  'report_dict': {'fact_dict': {'bias_rating': ' PSEUDOSCIENCE',
    'fact_report': ' LOW',
    'report_flag': True}

In [72]:
print(df_list)
print(df_list2)

[{'url': 'https://www.cnn.com', 'report_dict': {'fact_dict': {'bias_rating': ' RIGHT', 'fact_report': ' MIXED', 'report_flag': True}, 'error_url': 'https://www.cnn.com'}}, {'url': 'http://the-sun.co/', 'report_dict': {'fact_dict': {'bias_rating': None, 'fact_report': None, 'report_flag': False}, 'error_url': 'http://the-sun.co/'}}, {'url': 'https://www.nbcnews.com/', 'report_dict': {'fact_dict': {'bias_rating': ' LEAST BIASED', 'fact_report': ' MOSTLY FACTUAL', 'report_flag': True}, 'error_url': 'https://www.nbcnews.com/'}}, {'url': 'https://www.ntnews.com.au/', 'report_dict': {'fact_dict': {'bias_rating': ' RIGHT', 'fact_report': ' MIXED', 'report_flag': True}, 'error_url': 'https://www.ntnews.com.au/'}}, {'url': 'https://www.couriermail.com.au/', 'report_dict': {'fact_dict': {'bias_rating': ' LEAST BIASED', 'fact_report': ' MOSTLY FACTUAL', 'report_flag': True}, 'error_url': 'https://www.couriermail.com.au/'}}, {'url': 'https://www.theage.com.au/', 'report_dict': {'fact_dict': {'bias

In [74]:
urls_in_dataframe = list(df['url'])
report_dict_list = list(df['report_dict'])
for ind in range(len(urls_in_dataframe)):
  for item in df_list:
    if urls_in_dataframe[ind] == item['url']:
      report_dict_list[ind] = item['report_dict']
      break
  for item2 in df_list2:
    if urls_in_dataframe[ind] == item2['url']:
      report_dict_list[ind] = item2['report_dict']
      break




In [75]:
df_clean_dict = {'url': urls_in_dataframe,'report_dict': report_dict_list}
df_clean = pd.DataFrame(df_clean_dict)

In [81]:
from numpy.core.fromnumeric import nonzero
def extract_prep_row_for_csv(row):
  url = row['url']
  report = row['report_dict']
  biasdict = report['fact_dict']
  bias = None
  fact = None
  if biasdict is not None:
    bias = biasdict['bias_rating']
  factdict = report['fact_dict']
  if factdict is not None:
    fact = factdict['fact_report']
  repdict = {'url': url,'bias' : bias ,'fact' : fact}
  return repdict

csv_list = []
for index,row in df_clean.iterrows():
  report_dict = extract_prep_row_for_csv(row)
  csv_list.append(report_dict)






In [82]:
df_csv = pd.DataFrame(csv_list)


In [84]:
# Specify the file path where you want to save the CSV file
csv_file_path = '/content/sample_data/corpus.csv'

# Convert the DataFrame to a CSV file
df_csv.to_csv(csv_file_path, index=False)  # Setting index to False removes the index column

print(f'DataFrame has been saved to {csv_file_path}')

DataFrame has been saved to /content/sample_data/corpus.csv
